Indehold:

 - Assess your predicted strain designs using simulations and pathway visualizations
 - Assess manually derived strain designs using model simulations
 - Based on your computations, provide a top 10 list of most promising cell factory designs. The criteria for "most promising" are the number of modifications, yield, growth rate and others you might define

In [3]:
from cobra.io import read_sbml_model 
from cobra import Model, Reaction, Metabolite
import escher

In [4]:
model = read_sbml_model('models/iMM904_progesterone_fixed.xml') # Load model

In [5]:
model.add_boundary(model.metabolites.get_by_id("progesterone_c"), type="demand") # Because it is a reversible reaction, we need to set a demand for production of progesterone 

Reaction identifier,DM_progesterone_c
Name,progesterone demand
Memory address,0x13154cca0
Stoichiometry,progesterone_c --> progesterone -->
GPR,
Lower bound,0
Upper bound,1000.0


In [6]:
model2 = model.optimize()
model2.fluxes[model2.fluxes != 0.]

13GS        0.326670
ACCOAC      0.114455
AGAT_SC     0.007110
AGPRim      0.046260
AGTi        0.015001
             ...    
PSP_L       0.164049
PStm_SC     0.000030
URIDK2r    -0.001727
PYK        17.721990
PYRDC      15.946051
Name: fluxes, Length: 305, dtype: float64

In [8]:
escher.Builder('iMM904_progesterone_fixed.Central carbon metabolism',reaction_data = model2.fluxes.to_dict()).display_in_notebook()

Exception: Could not find the map iMM904_progesterone_fixed.Central carbon metabolism on the server